In [136]:
import boto3
import json
import pandas as pd
import requests

import sys
import os
import numpy as np
from scipy.optimize import fsolve


In [108]:
s3 = boto3.resource('s3')

listing = s3.Object('pb-get-listing', 'M3002977276.json')
listing_content = listing.get()['Body'].read().decode('utf-8')
listing_json = json.loads(listing_content)

In [109]:
def get_downpayment(json_file, down_pect=.2):
    price = json_file['price']
    json_file['down'] = price * down_pect
    json_file['loan'] = price - json_file['down']
    
    print(f"INFO: the {down_pect} downpayment on a property with price {price} is {json_file['down']}")
    
    return json_file

In [110]:
def get_closing_costs(price, pect_of_price=0.02):
    closing = price * pect_of_price
    print(f"INFO: closing costs are {pect_of_price} of {price} or {closing}")
    return closing
    

In [118]:
def get_monthly_loan_payment(loan_amount, int_rate=0.045, years=30):
	"""
	Calculates the monthly payment on a loan
	ARGUMENTS:
	--------------------------------------
	loan_amount (int): amount borrowed
	int_rate (float): yearly interest rate
	years (int): term of loan 
	RETURNS:
	--------------------------------------
	pmt_amount (singleton): monthly payment
	"""
	
	fv = 0
	pv = loan_amount
	int_rate = int_rate	
	years = years

	rate = int_rate / 12
	nper = years * 12
	

	def f(pmt):
		return fv + pv*(1 + rate)**nper + pmt*(1 + rate*0) / rate*((1 + rate)**nper - 1)
	
	pmt_amt = int(fsolve(f, [100], xtol=0.000001))

	print(f"INFO: calcualted payment amount of {pmt_amt}")


	return pmt_amt


In [119]:
def get_arv():
    return None

In [120]:
def get_repair_costs():
    return None

In [121]:
def get_property_growth():
    return None

In [129]:
def get_points(loan, points = 0.005):
    point_costs = loan * points
    print(f"INFO: the points on a loan of {loan} are {points} or {point_costs}")
    return point_costs

In [144]:
def get_rental_income(address, bathrooms, bedrooms, squarefootage, proptype="Single Family"):
    url = "https://realtymole-rental-estimate-v1.p.rapidapi.com/rentalPrice"

    querystring = {"propertyType":proptype,"address":address,"bathrooms":bathrooms,"compCount":"5","squareFootage":squarefootage,"bedrooms":bedrooms}

    headers = {
        'x-rapidapi-host': "realtymole-rental-estimate-v1.p.rapidapi.com",
        'x-rapidapi-key': "4519f6dcffmshfadff8b94661096p1989c5jsn14919517996b"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    rent = response.json()['rent']

    return rent

In [145]:
get_rental_income(address="296 Melita Road, Memphis TN", bathrooms=2, bedrooms=3, squarefootage=1400)

1570.54

In [147]:
listing_json['address']

{'city': 'Cleveland',
 'line': '529 E 112th St',
 'postal_code': '44108',
 'state_code': 'OH',
 'state': 'Ohio',
 'county': 'Cuyahoga',
 'fips_code': '39035',
 'county_needed_for_uniq': False,
 'lat': 41.542969,
 'lon': -81.606428,
 'neighborhood_name': 'Forest Hills'}

In [151]:
listing_json['address']['line'] + listing_json['address']['city']

'529 E 112th StCleveland'

In [130]:
listing_json = get_downpayment(listing_json, down_pect=.2)

INFO: the 0.2 downpayment on a property with price 32000 is 6400.0


In [131]:
listing_json['loan_payment'] = get_monthly_loan_payment(listing_json['loan'])

INFO: calcualted payment amount of -129


In [132]:
listing_json['loan_point_costs'] = get_points(listing_json['price']*.80, points = 0.005)

INFO: the points on a loan of 25600.0 are 0.005 or 128.0


In [133]:
listing_json['closing'] = get_closing_costs(listing_json['price'])

INFO: closing costs are 0.02 of 32000 or 640.0


In [134]:
listing_json

{'property_id': 'M3002977276',
 'listing_id': '2936207914',
 'products': ['core.agent', 'core.broker', 'co_broke'],
 'rdc_web_url': 'https://www.realtor.com/realestateandhomes-detail/529-E-112th-St_Cleveland_OH_44108_M30029-77276',
 'prop_type': 'single_family',
 'virtual_tour': {'href': 'https://www.propertypanorama.com/529-East-112th-St-Cleveland-OH-44108/unbranded'},
 'address': {'city': 'Cleveland',
  'line': '529 E 112th St',
  'postal_code': '44108',
  'state_code': 'OH',
  'state': 'Ohio',
  'county': 'Cuyahoga',
  'fips_code': '39035',
  'county_needed_for_uniq': False,
  'lat': 41.542969,
  'lon': -81.606428,
  'neighborhood_name': 'Forest Hills'},
 'branding': {'listing_office': {'list_item': {'name': 'RealHome Services Solutions',
    'photo': None,
    'phone': None,
    'slogan': None,
    'show_realtor_logo': False,
    'link': None,
    'accent_color': None}}},
 'prop_status': 'for_sale',
 'price': 32000,
 'baths_full': 1,
 'baths': 1,
 'beds': 4,
 'building_size': {'siz